# Modèle de Markov Caché du Premier Ordre

### Application à la correction de typos dans un texte

In [29]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys

UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)
                
                pair_counts is dictionary with 
                - key : a tuple (word,tag)
                - value: the associated count
                
                We just need to fill the matrix and normalize the count in the right way: 
                - one column (i constant) is the distrib. of word given a tag
                - just normalize the column, i.e sum over the row (axis=0)
            """
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0] # get word 
                tag=pair[1] # get tag  
                cpt=pair_counts[pair] # get the count
                # get word index (row), deal with OOV
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                # get tag  index (column)
                i=self.Y_index[tag]
                # fill the matrix
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
                
                trans_counts is dictionary with 
                - key : a tuple (tag-1,tag)
                - value: the associated count
                
                We just need to fill the matrix and normalize the count in the right way: 
                - one column (i constant) is the distrib. of tags for  given context tag-1
                - just normalize the column, i.e sum over the row (axis=0)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba[i,j]=trans_counts[pair]
            # normalize
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)
            
            
        def viterbi(self, wordids):
            """ Viterbi Algorithm : 
            Finding the most likely sequence of hidden states. 
            """
            T = len(wordids)

            # Initialization
            delta = zeros(self.N, float)
            psi = zeros((T, self.N), int)
            delta_t = zeros(self.N, float)
            tmp = zeros(self.N, float)

            # Apply initial_state probabilities to the first frame
            delta = self.observation_proba[wordids[0]] * self.initial_state_proba

            for t in xrange(1, T):
                O_t = wordids[t]
                for j in range(self.N):
                    multiply(delta, self.transition_proba[:, j], tmp)
                    idx = psi[t, j] = tmp.argmax()
                    delta_t[j] = tmp[idx] * self.observation_proba[O_t, j]
                delta, delta_t = delta_t, delta

            # Reconstruction
            i_star = [delta.argmax()]
            for psi_t in psi[-1:0:-1]:
                i_star.append(psi_t[i_star[-1]])
            i_star.reverse()
            return i_star
        
        def accuracy(self, test):
        
            nb_correct = 0
            total = 0
            after_hmm = 0
            m = 0

            for word in test:
                obsids, statids = self.data2indices(word)
                best_sequence = self.viterbi(obsids)

                # Correct caracters after Viterbi algorithm
                nb_correct += sum(np.array(best_sequence) == np.array(statids))

                # Number of typos after HMM correction
                after_hmm += sum(np.array(best_sequence) != np.array(statids))

                total += len(statids)

            print("Number of typos after correction : {}".format(after_hmm))
            
            accuracy = nb_correct * 100.0 / total
            print("Accuracy : {}% ".format(accuracy))
            return accuracy

# Compter les mots et les tags

In [30]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag) # (tag at t-1 , tag at t)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits


# Création du vocabulaire (filtrage selon le nombre d'occurence)

In [31]:
def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    voc.append(UNK)
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


# Les données


In [32]:
import pickle

# Données avec 10% de typos
train10 = pickle.load(open('typos-data/train10.pkl', 'rb'))
test10 = pickle.load(open('typos-data/test10.pkl', 'rb'))

# Données avec 20% de typos
train20 = pickle.load(open('typos-data/train20.pkl', 'rb'))
test20 = pickle.load(open('typos-data/test20.pkl', 'rb'))

In [33]:
"""
tot=len(data)
trainpart = 0.8
testpart  = 0.1
train=data[0:int(tot*trainpart)]
test= data[int(tot*trainpart)+1:int(tot*trainpart)+int(tot*testpart)]
print "Nombre de phrases totale = "+str(tot)
print "Nombre de phrases de train = "+str(len(train))
print "Nombre de phrases de test  = "+str(len(test))
"""

'\ntot=len(data)\ntrainpart = 0.8\ntestpart  = 0.1\ntrain=data[0:int(tot*trainpart)]\ntest= data[int(tot*trainpart)+1:int(tot*trainpart)+int(tot*testpart)]\nprint "Nombre de phrases totale = "+str(tot)\nprint "Nombre de phrases de train = "+str(len(train))\nprint "Nombre de phrases de test  = "+str(len(test))\n'

In [34]:
train = train10

cwords,ctags,cpairs,ctrans,cinits = make_counts(train)
print "Nombre de lettres  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))
print ctags
vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de lettres  : 26
Nombre de tags  : 26
Nombre de paires: 127
Nombre de trans : 403 / 144
Nombre de init. : 25
{'a': 10560, 'c': 4808, 'b': 2070, 'e': 18091, 'd': 4541, 'g': 2736, 'f': 3379, 'i': 10976, 'h': 6683, 'k': 590, 'j': 108, 'm': 3773, 'l': 6417, 'o': 11935, 'n': 9778, 'q': 150, 'p': 3217, 's': 9762, 'r': 8247, 'u': 3931, 't': 13877, 'w': 2229, 'v': 1927, 'y': 2985, 'x': 274, 'z': 124}
Vocabulaire :27


# Création du HMM

In [35]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None)

HMM creating with: 
26 states
27 observations


# Apprentissage pas à pas 

In [36]:
hmm.observation_estimation(cpairs)
print hmm.observation_proba.sum(axis=0)


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]


In [37]:
hmm.transition_estimation(ctrans)
print hmm.transition_proba.sum(axis=0)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]


In [38]:
hmm.init_estimation(cinits)
print sum(hmm.initial_state_proba)

1.0


# Apprentissage en une fois

In [39]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 0.001)
hmm.supervised_training(cpairs,ctrans,cinits)

HMM creating with: 
26 states
27 observations


# Tests

In [40]:
hmm.accuracy(test10)

Number of typos after correction : 1426
Accuracy : 80.5191256831% 


80.519125683060111